# 这是一个演示Towhee流水线工程的示例项目

## 组件
向量数据库 : milvus 

## Example 1
一个简单的towhee pipe演示程序，将输入数字增加1后输出。

## Example 2
演示如何将一个图片转换为向量，存入milvus向量数据库

## Example 3 
演示了sentence-t5-xxl 使用towhee pipe计算embedding，得到中文句子上的相似性计算结果

In [ ]:
!pip install -r requirements.txt

In [ ]:
milvus_host = '127.0.0.1'
milvus_port = '19530'

In [58]:
# 列出所有towhee支持的model
from towhee import ops

op = ops.sentence_embedding.transformers().get_op()
full_list = op.supported_model_names()
onnx_list = op.supported_model_names(format = 'onnx')

print(full_list)
print(onnx_list)

['all-MiniLM-L12-v1', 'all-MiniLM-L12-v2', 'all-MiniLM-L6-v1', 'all-MiniLM-L6-v2', 'all-distilroberta-v1', 'all-mpnet-base-v1', 'all-mpnet-base-v2', 'all-roberta-large-v1', 'bert-base-nli-mean-tokens', 'bert-base-uncased', 'bert-large-uncased', 'bert-large-uncased-whole-word-masking', 'distilbert-base-uncased', 'distiluse-base-multilingual-cased-v1', 'distiluse-base-multilingual-cased-v2', 'facebook/bart-large', 'gpt2-xl', 'microsoft/deberta-xlarge', 'microsoft/deberta-xlarge-mnli', 'msmarco-bert-base-dot-v5', 'msmarco-distilbert-base-tas-b', 'msmarco-distilbert-base-v4', 'msmarco-distilbert-dot-v5', 'multi-qa-MiniLM-L6-cos-v1', 'multi-qa-MiniLM-L6-dot-v1', 'multi-qa-distilbert-cos-v1', 'multi-qa-distilbert-dot-v1', 'multi-qa-mpnet-base-cos-v1', 'multi-qa-mpnet-base-dot-v1', 'paraphrase-MiniLM-L12-v2', 'paraphrase-MiniLM-L3-v2', 'paraphrase-MiniLM-L6-v2', 'paraphrase-TinyBERT-L6-v2', 'paraphrase-albert-small-v2', 'paraphrase-distilroberta-base-v2', 'paraphrase-mpnet-base-v2', 'paraphra

In [ ]:
# Example 1
from towhee import pipe

add_one = (
    pipe.input('x')
        .map('x', 'y', lambda x: x+1)
        .output('y')
)

res = add_one(0).get()
print(res)

In [ ]:
# Example 2
from towhee import pipe, ops

img_embedding = (
    pipe.input('url')
        .map('url', 'img', ops.image_decode.cv2())
        .map('img', 'embedding', ops.image_embedding.timm(model_name = 'resnet50'))
        .output('embedding')
)

url = 'pic/test1.jpg'
res = img_embedding(url).get()

In [ ]:
from pymilvus import Collection, utility, connections
import pandas as pd
collection_name = 'pcapel_towhee'
print(res)
# 假定已经创建milvus collection：pcapel_towhee
connections.connect(host=milvus_host, port=milvus_port)
collection = Collection(collection_name)

df = pd.DataFrame({'id':[1], 'name':['test'], 'embedding':[res[0]]})
# print(dict1)
collection.insert( df.to_dict(orient = 'records'))

In [28]:
# Example 3 
from towhee import pipe, AutoConfig, AutoPipes
import torch
import torch.nn.functional as F
import time
import threading
import multiprocessing
from concurrent.futures import ThreadPoolExecutor

auto_config = AutoConfig.load_config('sentence_embedding')
## https://huggingface.co/sentence-transformers/sentence-t5-xxl
auto_config.model = 'sentence-t5-xxl'
auto_config.device = -1

sentence_embedding = AutoPipes.pipeline('sentence_embedding', config=auto_config)


def start_thread_embedding(text): 
    return torch.tensor(sentence_embedding(text).get(), dtype=torch.float)

In [8]:
# 中文 Sentence Token
text1 = '随着科技的快速发展，人工智能在我们的生活中扮演着越来越重要的角色，从智能家居到自动驾驶汽车，它的应用领域日益广泛。'
text2 = '科技进步带动了人工智能技术的飞速发展，它已经深入到我们生活的各个方面，包括智能家电和自动化交通工具，成为不可或缺的一部分。'

# embedding1 = start_thread_embedding('随着科技的快速发展，人工智能在我们的生活中扮演着越来越重要的角色，从智能家居到自动驾驶汽车，它的应用领域日益广泛。')
# embedding2 = start_thread_embedding('科技进步带动了人工智能技术的飞速发展，它已经深入到我们生活的各个方面，包括智能家电和自动化交通工具，成为不可或缺的一部分。')
# with ThreadPoolExecutor(max_workers=2) as executor:
#     embedding1 = executor.submit(start_thread_embedding, text1).result()
#     embedding2 = executor.submit(start_thread_embedding, text2).result()
# executor = ThreadPoolExecutor(max_workers=5)
# for result in executor.map(start_thread_embedding, [text1, text2]):
#     print(f"完成向量化")
# executor.shutdown()
# start_time = time.time()
# print("Number of cpu : ", multiprocessing.cpu_count())
# p = multiprocessing.Process(target=print_text, args=(text1,))
# p.start()
# p.join()
end_time = time.time()
print(f"执行时长: {end_time - start_time:.4f} 秒")

# consin_sim = F.cosine_similarity(embedding1.float(), embedding2.float())
# print(consin_sim.item())
# batch generate embeddings for multi-sentences
# embeddings = sentence_embedding.batch(['how are you?', 'how old are you?'])


Number of cpu :  12
执行时长: 0.0827 秒


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/anaconda3/envs/data-processing-v2/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/anaconda3/envs/data-processing-v2/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'print_text' on <module '__main__' (built-in)>


In [27]:
# import threading
from concurrent.futures import ThreadPoolExecutor

def print_text(str = 'anon'):
    print(f"{str}")
    return str


# threads = []
# thread = threading.Thread(target=print_text, args=('anon1',))
# threads.append(thread)
# thread.start()
# thread.join()


# 创建线程池
with ThreadPoolExecutor(max_workers=4) as pool:
    # 提交任务
    results = pool.map(print_text, ('test', 'test2'))

# 打印结果
for r in results:
    print(f"{r}")

test
test2
test
test2
